# Face Mask Detection

## Introduction

In this notebook, I'll show you how to create a custom dataset for both YOLO-based object detection and Vision Transformer (ViT) Transformers-based face mask classification using the face mask detection dataset from Kaggle. This dataset contains images of people with, without masks, and mask weared incorrect, along with annotations specifying the location of faces and labels indicating mask-wearing status.

We'll explore two different approaches to utilize this dataset: one for YOLO-based object detection and another for ViT Transformers-based face mask classification. YOLO-based object detection will enable us to detect and localize faces with masks, while ViT Transformers will allow us to classify whether a person is wearing a mask or not.

## Setup

First, make sure you have all the necessary libraries installed. These include; 
- pandas
- matplotlib==3.8.4
- scikit-learn==1.4.2
- pyyaml==6.0.1
- pillow==10.3.0
- ultralytics==8.0.208
- transformers==4.40.1
- datasets==2.19.1

Here to install the libraries,
```
!pip install pandas scikit-learn==1.4.2 pyyaml==6.0.1 pillow==10.3.0 ultralytics==8.0.208 transformers==4.40.1 datasets==2.19.1

```

In [1]:
import os
import glob
import shutil
import random
import xml.etree.ElementTree as ET
import yaml
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

## Dataset Preparation

1. Download the face mask detection dataset from [Kaggle](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection/).
2. Extract the dataset files into a directory structure containing images and annotations.

### Download face mask detection dataset

```
!wget -o face_mask_dataset.zip https://www.kaggle.com/datasets/andrewmvd/face-mask-detection/download?datasetVersionNumber=1
```

### Extract downloaded dataset

```
unzip face_mask_dataset.zip -d face_mask_dataset
```

### Dataset Generator

Here's breakdown of the process:
1. Dataset Loading and Preparation:
- The code loads image files and corresponding annotations from the dataset directory.
- It extracts image paths and annotation paths from the specified directory structure.
- The dataset is organized into images and annotations, which are used for further processing.
2. Data Transformation and Annotation Handling:
- The code processes annotations to extract bounding box coordinates and class labels for each object (e.g., face with or without a mask).
- Image annotations are converted into YOLO format, which includes class labels and bounding box coordinates normalized to image dimensions.
3.Dataset Splitting:
- The dataset is split into training, validation, and test sets using the train_test_split.
- The splitting process ensures that images and corresponding annotations are kept together in their respective sets.
4. Dataset Saving:
- The processed dataset is saved in a directory structure suitable for YOLO-based object detection.
- Images are saved in the images directory, and annotations are saved in the labels directory, with corresponding file names.
5. Configuration File Creation:
- A configuration file is generated to specify the paths to the train, validation, and test sets, along with other parameters required for model training.
- This configuration file is saved in YAML format and contains essential information for training the YOLO model.

In [2]:
class DatasetGenerator:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.images_path = self._load_images()
        self.annotations_path = self._load_annotations()
        self.labels_dict = {
            "with_mask": 0,
            "without_mask": 1,
            "mask_weared_incorrect": 2,
        }
        self.images, self.labels = self._extract_dataset_(self.annotations_path)

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, index: int) -> tuple:
        image = self.images[index]
        label = self.labels[index]
        return image, label

    def to_dict(self) -> dict:
        return dict(image=self.images, labels=self.labels)

    def _extract_dataset_(self, annotation_path: list[str])->tuple[list, list]:
        images = []
        labels = []
        for annotation in annotation_path:
            data = self._get_annotations(annotation)
            image = Image.open(data["image_path"]).convert("RGB")
            for bbox, label in zip(data["bbox"], data["labels"]):
                roi = image.crop(bbox)
                images.append(roi)
                labels.append(label)

        return images, labels

    def _load_images(self)->list[str]:
        return glob.glob(os.path.join(self.data_dir, "images", "*.*"))

    def _load_annotations(self)->list[str]:
        return glob.glob(os.path.join(self.data_dir, "annotations", "*.xml"))

    def _convert_coordinates(self, size: tuple, box: tuple) -> list[float]:
        xmin, ymin, xmax, ymax = box
        dw, dh = 1.0 / size[0], 1.0 / size[1]
        x = xmin
        y = ymin
        w = xmax - xmin
        h = ymax - ymin
        x *= dw
        y *= dh
        w *= dw
        h *= dh
        return [x, y, w, h]

    def _get_annotations(self, annotation_path: str) -> dict:
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        image_dir = root.find("folder").text
        filename = root.find("filename").text
        image_path = os.path.join(self.data_dir, image_dir, filename)
        size = root.find("size")
        width, height = int(size.find("width").text), int(size.find("height").text)
        bboxes, labels = [], []
        yolo_annotations = []

        for obj in root.findall("object"):
            label = obj.find("name").text
            label_id = self.labels_dict[label]
            bbox = obj.find("bndbox")
            xmin, ymin, xmax, ymax = (
                int(bbox.find(c).text) for c in ["xmin", "ymin", "xmax", "ymax"]
            )
            box = (xmin, ymin, xmax, ymax)
            bboxes.append(box)
            labels.append(label_id)

            xywh_box = self._convert_coordinates((width, height), box)
            xywh_box = " ".join([str(item) for item in xywh_box])

            yolo_annotations.append([str(label_id), xywh_box])

        return {
            "image_path": image_path,
            "bbox": bboxes,
            "labels": labels,
            "size": (width, height),
            "yolo_annotations": yolo_annotations,
        }

    def _save_annotations(self, annotations: list, filepath: str) -> None:
        with open(filepath, "w") as f:
            for annotation in annotations:
                line = " ".join([str(ann) for ann in annotation])
                f.write(line + "\n")

    def create_dataset_yolo(self, save_dir: str, config_file: str):
        train, val, test = self.split_dataset()
        train_annotations = [
            self._get_annotations(annotation_path)["yolo_annotations"]
            for annotation_path in train[1]
        ]

        val_annotations = [
            self._get_annotations(annotation_path)["yolo_annotations"]
            for annotation_path in val[1]
        ]
        test_annotations = [
            self._get_annotations(annotation_path)["yolo_annotations"]
            for annotation_path in test[1]
        ]

        def _save_dataset(data: tuple, split: str):
            dataset_dir = os.path.join(save_dir, split)
            os.makedirs(dataset_dir, exist_ok=True)
            images_dir = os.path.join(dataset_dir, "images")
            os.makedirs(images_dir, exist_ok=True)
            annotations_dir = os.path.join(dataset_dir, "labels")
            os.makedirs(annotations_dir, exist_ok=True)
            for i, (image_path, annotation) in enumerate(zip(data[0], data[1])):
                image_name = image_path.split("/")[-1]
                annotation_name = image_name[:-3] + "txt"
                shutil.copy(image_path, os.path.join(images_dir, image_name))
                print(f'Successfully created {os.path.join(images_dir, image_name)}')
                self._save_annotations(
                    annotation, os.path.join(annotations_dir, annotation_name)
                )
                print(f'Successfully created {os.path.join(annotations_dir, annotation_name)}')
                

        _save_dataset((train[0], train_annotations), "train")
        _save_dataset((val[0], val_annotations), "val")
        _save_dataset((test[0], test_annotations), "test")

        config = {
            "train": os.path.join(os.getcwd(), save_dir, "train", "images"),
            "val": os.path.join(os.getcwd(), save_dir, "val", "images"),
            "test": os.path.join(os.getcwd(), save_dir, "test", "images"),
            "nc": 3,
            "names": ["with_mask", "without_mask", "mask_weared_incorrect"],
        }

        with open(f"{save_dir}/{config_file}.yaml", "w") as yaml_file:
            yaml.dump(config, yaml_file, default_flow_style=False, sort_keys=False)
            print(f'Successfully created data config file {save_dir}/{config_file}.yaml')

    def split_dataset(self, split_size: float = 0.1, random_seed: int = 42) -> tuple:
        train_images, val_images, train_annotations, val_annotations = train_test_split(
            self.images_path, self.annotations_path, test_size=split_size, random_state=random_seed
        )
        train_images, test_images, train_annotations, test_annotations = train_test_split(
            train_images, train_annotations, test_size=split_size, random_state=random_seed
        )

        return (
            (train_images, train_annotations),
            (val_images, val_annotations),
            (test_images, test_annotations),
        )

## Face Mask Detection Using YOLOv8

### Import the library **Ultralytics**

In [3]:
from ultralytics import YOLO

### Generate and create dataset using class DatasetGenerator

In [4]:
DATASET_DIR = 'face_mask_dataset'

In [5]:
# dataset = DatasetGenerator(DATASET_DIR)
# dataset.create_dataset_yolo(save_dir='face_mask_dataset_yolo', config_file='data')

### Model Development

In [6]:
class YOLOTrainer:
    def __init__(self, model_path: str, data_path: str, project_name: str, hyp_path: str):
        self.model_path = model_path
        self.hyp_path = hyp_path
        self.data_path = data_path
        self.project_name = project_name
        self.hyperparameters = None
        self.model = None

    def load_model(self):
        self.model = YOLO(self.model_path)

    def load_hyperparameters(self):
        with open(self.hyp_path, 'r') as f:
            hyperparameters = yaml.safe_load(f)
        self.hyperparameters = hyperparameters

    def train(self, epochs: int, batch_size: int, optimizer: str = 'SGD', pretrained: bool = True):
        self.model.train(
            data=self.data_path,
            epochs=epochs,
            batch=batch_size,
            project=self.project_name,
            optimizer=optimizer,
            pretrained=pretrained,
            ** self.hyperparameters
        )

    def evaluate(self, validation_data: str):
        metrics = self.model.val(data=validation_data)
        return metrics

    def save_model(self, output_path: str):
        self.model.save(output_path)

### Model Training

In [7]:
trainer = YOLOTrainer(data_path='./yolo_format/data.yaml', model_path='yolov8n.pt', hyp_path='hyperparameters.yaml', project_name='face_mask_detection_yolov8')

In [8]:
trainer.load_model()

In [9]:
trainer.load_hyperparameters()

In [10]:
trainer.train(epochs=100, batch_size=64, optimizer='SGD')

New https://pypi.org/project/ultralytics/8.2.13 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.11 🚀 Python-3.10.4 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16079MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./yolo_format/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=face_mask_detection_yolov8, name=train6, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=15, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

/home/aidev6-pc/anaconda3/envs/face_mask_det.python3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AMP: checks passed ✅


/home/aidev6-pc/anaconda3/envs/face_mask_det.python3.10/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
train: Scanning /home/aidev6-pc/dev/bimo/face_mask_detection/yolo_format/train/labels... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:00<00:00, 1808.37it/s]

train: New cache created: /home/aidev6-pc/dev/bimo/face_mask_detection/yolo_format/train/labels.cache


...


100 epochs completed in 0.067 hours.
Optimizer stripped from face_mask_detection_yolov8/train6/weights/last.pt, 6.3MB
Optimizer stripped from face_mask_detection_yolov8/train6/weights/best.pt, 6.3MB

Validating face_mask_detection_yolov8/train6/weights/best.pt...
Ultralytics YOLOv8.2.11 🚀 Python-3.10.4 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16079MiB)


libpng warning: iCCP: Not recognizing known sRGB profile that has been edited


Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


/home/aidev6-pc/anaconda3/envs/face_mask_det.python3.10/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.28it/s]
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited


                   all         86        535       0.86      0.764      0.838      0.553
             with_mask         86        464      0.932      0.914      0.944      0.641
          without_mask         86         55       0.75      0.818      0.857      0.521
 mask_weared_incorrect         86         16      0.899      0.559      0.713      0.496
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to face_mask_detection_yolov8/train6


### Model Evaluation

In [12]:
metrics = trainer.evaluate(validation_data='./yolo_format/data.yaml')

Ultralytics YOLOv8.2.11 🚀 Python-3.10.4 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16079MiB)


val: Scanning /home/aidev6-pc/dev/bimo/face_mask_detection/yolo_format/val/labels.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
/home/aidev6-pc/anaconda3/envs/face_mask_det.python3.10/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self

                   all         86        535      0.862      0.764      0.834      0.547
             with_mask         86        464      0.934      0.916      0.944       0.64
          without_mask         86         55      0.752      0.818      0.854      0.516
 mask_weared_incorrect         86         16      0.899      0.558      0.705      0.485
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to face_mask_detection_yolov8/train63


If we see the result of model evaluation, we got some metrics.
1. **Box Precision (P)**
- The precision metric for the bounding box predictions. It measures the accuracy of the detected boxes in terms of how many detected boxes are actually correct. Then we got, a precision of 0.934 for "with_mask" means that 93.4% of the boxes around faces with masks are correctly placed.
![Precision(P)](face_mask_detection_yolov8/train63/P_curve.png)

2. **Recall (R)**
- This metric measures the model's ability to find all the relevant cases (instances) within the datasets. For "without_mask", a recall of 0.818 means the model correctly identifies 81.8% of all actual "without_mask" instances in the test set.
![Recall(R)](face_mask_detection_yolov8/train63/R_curve.png)

3. **mAP50**
- Mean Average Precision at 50% IoU (Intersection over Union). This is a common metric for evaluating object detection models, where a detection is considered correct if the IoU is above 50%. The mAP50 of 0.944 for "with_mask" is particularly high, indicating excellent model performance for this class at this IoU threshold.

4. **mAP50-95**
- This is an average of the mAP calculated at different IoU thresholds from 50% to 95% (inclusive), in steps of 5%. It is a more stringent measure since it considers a range of IoU thresholds. The value of 0.64 for "with_mask" under this metric shows the model performs well even under stricter criteria, though naturally, the score is lower than the mAP50. You can see the results, in this figure below:
![Results](face_mask_detection_yolov8/train6/results.png)

### Conclusion

1. **Overall Performance**:
The model performs very well on detecting faces with masks, with high precision and mAP scores, which because that the training data likely had a large number of "with_mask" examples, or the model features are particularly good at picking out this class.

2. **Model Robustness**: The model is generally robust, as indicated by consistent training and validation loss curves, and maintains good precision across high confidence levels.

3. The model shows excellent precision and recall for faces with masks.

4. The model achieves commendable mAP50 and mAP50-95 scores, especially in the "with_mask" category, 

5. The model struggle slightly with detecting "mask_weared_incorrect", as indicated by the lower recall and mAP scores.



### WHAT SHOULD WE DO?

1. Add more images that represent diverse scenarios, especially focusing on underrepresented classes like "without_mask" and "mask_weared_incorrect". Make sure to include a variety of backgrounds, lighting conditions, and face orientations.

2. Apply transformations such as rotations, scaling, cropping, and color adjustments to existing images to create a more robust training dataset. This helps the model generalize better across different real-world conditions.

3. Adjust Hyperparameters: Optimize learning rates, decay schedules, and other hyperparameters. Sometimes, small adjustments can yield significant improvements.

## Face Mask Detection (using Classification)

### Import libraries needed

In [ ]:
import torch
from transformers import (
    ViTImageProcessor,
    ViTModel,
    ViTForImageClassification,
    Trainer,
    TrainingArguments
)

from torchvision.transforms import (
    Compose,
    Normalize,
    RandomResizedCrop,
    RandomHorizontalFlip,
    CenterCrop,
    Resize,
    ToTensor,
)

from sklearn.utils.class_weight import compute_sample_weight
from datasets import Dataset, DatasetDict, load_metric, ClassLabel


#### Initialize parameters

In [ ]:
DATA_DIR = "./face_mask_dataset/"
MODEL_CHECKPOINT = "google/vit-base-patch16-224-in21k"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_OUTPUT = "vit-face-mask-classification"
NUM_OF_EPOCHS = 10
BATCH_SIZE = 64
STRATEGY = "epoch"
LR = 5e-5

### Generate Data from DatasetGenerator

In [ ]:
DATA_DIR = './face_mask_dataset'

In [ ]:
dataset = DatasetGenerator(DATA_DIR)

In [ ]:
dataset.to_dict()

{'image': [<PIL.Image.Image image mode=RGB size=30x37>,
  <PIL.Image.Image image mode=RGB size=41x44>,
  <PIL.Image.Image image mode=RGB size=35x51>,
  <PIL.Image.Image image mode=RGB size=33x35>,
  <PIL.Image.Image image mode=RGB size=37x35>,
  <PIL.Image.Image image mode=RGB size=16x23>,
  <PIL.Image.Image image mode=RGB size=31x41>,
  <PIL.Image.Image image mode=RGB size=21x30>,
  <PIL.Image.Image image mode=RGB size=30x26>,
  <PIL.Image.Image image mode=RGB size=23x20>,
  <PIL.Image.Image image mode=RGB size=29x29>,
  <PIL.Image.Image image mode=RGB size=28x33>,
  <PIL.Image.Image image mode=RGB size=96x116>,
  <PIL.Image.Image image mode=RGB size=56x58>,
  <PIL.Image.Image image mode=RGB size=13x21>,
  <PIL.Image.Image image mode=RGB size=45x39>,
  <PIL.Image.Image image mode=RGB size=116x106>,
  <PIL.Image.Image image mode=RGB size=39x49>,
  <PIL.Image.Image image mode=RGB size=52x56>,
  <PIL.Image.Image image mode=RGB size=48x60>,
  <PIL.Image.Image image mode=RGB size=42x42>,
 

### Preprocessing Data for Training Model

In [ ]:
id2label = dataset.labels_dict
label2id = {v: k for k, v in id2label.items()}

In [ ]:
ds = Dataset.from_dict(dataset.to_dict())
ds = ds.cast_column("labels", ClassLabel(names=list(id2label.keys())))

Casting the dataset: 100%|██████████| 4072/4072 [00:00<00:00, 268532.53 examples/s]


#### Splitting Dataset

In [ ]:
# Splitting dataset into train and val
ds = ds.train_test_split(test_size=0.1, shuffle=True, seed=42)
ds = DatasetDict({"train": ds["train"], "val": ds["test"]})

#### Transformation Dataset

In [ ]:
def preprocess_image(example_batch, transforms):
    example_batch["pixel_values"] = [
        transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

In [ ]:
processor = ViTImageProcessor.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
image_transforms = Compose(
    [
        Resize((processor.size["height"], processor.size["width"])),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(
            mean=processor.image_mean,
            std=processor.image_std,
        ),
    ]
)

In [ ]:
# Applied transform
ds["train"].set_transform(lambda x: preprocess_image(x, image_transforms))
ds["val"].set_transform(lambda x: preprocess_image(x, image_transforms))

### Training Model

In [ ]:
def compute_metrics(p):
    accuracy_metric = load_metric("accuracy")
    accuracy = accuracy_metric.compute(
        predictions=np.argmax(p.predictions, axis=1), references=p.label_ids
    )["accuracy"]
    f1_score_metric = load_metric("f1")
    weighted_f1_score = f1_score_metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids,
        average="weighted",
    )["f1"]
    recall_metric = load_metric("recall")
    weighted_recall = recall_metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids,
        average="weighted",
    )["recall"]
    precision_metric = load_metric("precision")
    weighted_precision = precision_metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids,
        average="weighted",
    )["precision"]
    return {
        "accuracy": accuracy,
        "Weighted F1": weighted_f1_score,
        "Weighted Recall": weighted_recall,
        "Weighted Precision": weighted_precision,
    }

In [ ]:
def collate_fn(example_batch):
    pixel_values = torch.stack([example["pixel_values"] for example in example_batch])
    labels = torch.tensor([example["labels"] for example in example_batch])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
model = ViTForImageClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)
model.to(DEVICE)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [ ]:
args = TrainingArguments(
    MODEL_OUTPUT,
    remove_unused_columns=False,
    evaluation_strategy=STRATEGY,
    save_strategy=STRATEGY,
    logging_strategy="steps",
    logging_steps=10,
    logging_first_step=True,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    num_train_epochs=NUM_OF_EPOCHS,
    warmup_ratio=0.10,
    save_total_limit=1,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["val"],
    tokenizer=processor,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [ ]:
train_results = trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

eval_results = trainer.evaluate(ds["val"])
trainer.log_metrics("eval", eval_results.metrics)
trainer.save_metrics("eval", eval_results.metrics)
trainer.save_state()

### Model Evaluation

#### Import libraries needed

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

### Load Trained Model

In [ ]:
model = ViTForImageClassification.from_pretrained(MODEL_OUTPUT)
processor = ViTImageProcessor.from_pretrained(MODEL_OUTPUT)

### Generate Data for Evaluation

In [ ]:
dataset = DatasetGenerator(DATASET_DIR)

In [ ]:
results = []

data = {
    "ground_truth": [],
    "label": [],
    "label_name": [],
    "probs": [],
}

for image, label in dataset:
    ground_truth = label
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)

    predicted_class_idx = probabilities.argmax(-1).item()
    predicted_class = model.config.id2label[predicted_class_idx]

    results.append(predicted_class_idx)

    data["ground_truth"].append(ground_truth)
    data["label"].append(predicted_class_idx)
    data["label_name"].append(predicted_class)
    data["probs"].append(probabilities.max().detach().cpu().numpy())

In [ ]:
df = pd.DataFrame(data)
df.head()

,ground_truth,label,label_name,probs
0,1,1,without_mask,0.94091743
1,0,0,with_mask,0.96266997
2,1,1,without_mask,0.9440221
3,0,0,with_mask,0.9663677
4,0,0,with_mask,0.9665936


In [ ]:
df.to_csv('results.csv', index=True)

In [ ]:
y_true = df["ground_truth"].values
y_pred = df["label"].values

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")
# print(classification_report(y_true, y_pred))
# print(confusion_matrix(y_true, y_pred))

Accuracy: 0.9912


#### Visualize

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
plot_confusion_matrix(y_true, y_pred)

<Figure size 800x600 with 1 Axes>